# COVID-19 AUSTRIA MAP CASES

Imports

In [1]:
import sys
import os
from osgeo import ogr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from qgis.core import *
from qgis.PyQt.QtGui import *
from qgis.PyQt.QtCore import *
from IPython.display import Image

Start Qgis with GUI and start a project

In [2]:
qgis_path = r"C:\OSGeo4W64\apps\qgis"
QgsApplication.setPrefixPath(qgis_path, True)
qgs = QgsApplication([], True)
qgs.initQgis()
sys.path.append(r"C:\OSGeo4W64\apps\qgis\python\plugins")
project = QgsProject.instance()
print(QgsApplication.showSettings())

Application state:
QGIS_PREFIX_PATH env var:		C:/OSGEO4~1/apps/qgis
Prefix:		C:/OSGEO4~1/apps/qgis
Plugin Path:		C:/OSGEO4~1/apps/qgis/plugins
Package Data Path:	C:/OSGEO4~1/apps/qgis/.
Active Theme Name:	
Active Theme Path:	C:/OSGEO4~1/apps/qgis/./resources/themes\\icons/
Default Theme Path:	:/images/themes/default/
SVG Search Paths:	C:\OSGeo4W64\apps\qgis/./svg/
		C:/OSGEO4~1/apps/qgis/./svg/
		C:/Users/Marcelo/AppData/Roaming/python3\profiles\default/svg/
User DB Path:	C:/OSGEO4~1/apps/qgis/./resources/qgis.db
Auth DB Path:	C:/Users/Marcelo/AppData/Roaming/python3\profiles\default/qgis-auth.db



Download Map

In [3]:
#import urllib.request
#url = 'https://data.biogeo.ucdavis.edu/data/diva/adm/AUT_adm.zip'
#filename = 'map/AUT_adm.zip'
#urllib.request.urlretrieve(url, filename)

Unzip Map

In [4]:
import zipfile
filename = 'map/AUT_adm.zip'
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('map')

Load map layer

In [5]:
path = "map/AUT_adm1.shp"
map_layer = QgsVectorLayer(path, 'AUSTRIA map', 'ogr')

#pr = map_layer.dataProvider() # need to create a data provider
#pr.addAttributes([QgsField("",  QVariant.Int)]) # define/add field data type
#map_layer.updateFields()

if not map_layer.isValid():
    print("Failed to load the layer!")
else:
    project.addMapLayer(map_layer)
    print("Sucess")

Sucess


Download csv file

In [6]:
import urllib.request
url = 'https://raw.githubusercontent.com/covid19-eu-zh/covid19-eu-data/master/dataset/covid-19-at.csv'
filename = 'covid_data/covid-19-at.csv'
urllib.request.urlretrieve(url, filename)

('covid_data/covid-19-at.csv', <http.client.HTTPMessage at 0x18ad9d49ef0>)

Get most recent data from csv file

In [7]:
import csv
import datetime
from datetime import date

today = date.today()
today = today.strftime("%y-%m-%d")
print("Today's date:", today)

safe_date = date.today()-datetime.timedelta(1)
safe = safe_date.strftime("%y-%m-%d")
print("Safe date:", safe)

date_time_str = '2018-06-29T08:15:27'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S')

with open('covid_data/covid-19-at.csv', 'r') as i, open('covid_data/covid-19-austria.csv', 'w', newline='') as o:
    r = csv.reader(i, delimiter=',')
    writer = csv.writer(o)
    writer.writerow(["country", "name", "cases", "recovered", "deaths", "tests", "hospitalized", "intensive_care", "datetime"])
    #for row in r:
    #    if row[8] == "2020-05-09T15:00:00": 
    #        writer.writerow(row)
    #        print(row)
    max_value = date_time_obj
    #most recent date
    for row in r:
        if today in row[8]:
            date_time = row[8]
            date_time_object = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%M:%S')
            if date_time_object > max_value: 
                max_value = date_time_object
    print("Most recent date: ", max_value)
    
                
                
                
with open('covid_data/covid-19-at.csv', 'r') as i, open('covid_data/covid-19-austria.csv', 'w', newline='') as o:
    r = csv.reader(i, delimiter=',')
    writer = csv.writer(o)
    writer.writerow(["country", "name", "cases", "recovered", "deaths", "tests", "hospitalized", "intensive_care", "datetime"])
                
    #create csv with most recent data           
    for row in r:
        if row[1] != '':
            if today in row[8]:
                date_time = row[8]
                date_time_object = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%M:%S')
                if date_time_object == max_value: 
                    writer.writerow(row)
                    #print(row) 
        
pd.read_csv('covid_data/covid-19-austria.csv')
        

Today's date: 20-05-30
Safe date: 20-05-29
Most recent date:  2020-05-30 18:00:00


,country,name,cases,recovered,deaths,tests,hospitalized,intensive_care,datetime
0,AT,Burgenland,342,325,11,13058,2,0,2020-05-30T18:00:00
1,AT,Kärnten,414,398,13,16935,0,0,2020-05-30T18:00:00
2,AT,Vorarlberg,898,866,19,21298,0,0,2020-05-30T18:00:00
3,AT,Salzburg,1220,1196,38,33149,0,0,2020-05-30T18:00:00
4,AT,Steiermark,1836,1642,147,50347,6,0,2020-05-30T18:00:00
5,AT,Oberösterreich,2295,2214,60,62343,7,0,2020-05-30T18:00:00
6,AT,Niederösterreich,2866,2667,100,68149,25,10,2020-05-30T18:00:00
7,AT,Wien,3219,2793,172,104094,46,10,2020-05-30T18:00:00
8,AT,Tirol,3537,3419,108,72770,12,3,2020-05-30T18:00:00


Load csv file

In [8]:
csv_path = "file:covid_data/covid-19-austria.csv?delimiter=(',')"
csv_layer = QgsVectorLayer(csv_path, 'Data', 'delimitedtext')

if not csv_layer.isValid():
    print('Layer failed to load!')
else:
    project.addMapLayer(csv_layer)
    print("Sucess")

Sucess


Join layer

In [9]:
joinName = 'name'
targetName = 'NAME_1'

joinObject = QgsVectorLayerJoinInfo()
joinObject.setJoinFieldName(joinName)
joinObject.setTargetFieldName(targetName)
joinObject.setJoinLayerId(csv_layer.id())
joinObject.setUsingMemoryCache(True)
joinObject.setJoinLayer(csv_layer)
flag = map_layer.addJoin(joinObject)

In [10]:
import datetime
from datetime import date

today = date.today()
today = today.strftime("%m/%d/%Y")

safe_date = date.today()-datetime.timedelta(2)
safe = safe_date.strftime("%Y-%m-%d")
print("tests by state at:", today)

tests by state at: 05/30/2020


In [11]:
cases_layer = map_layer.clone()
cases_layer.setName("Number of confirmed tests")

for field in cases_layer.fields():
    print("Column "+field.name()+" Load")

Column ID_0 Load
Column ISO Load
Column NAME_0 Load
Column ID_1 Load
Column NAME_1 Load
Column TYPE_1 Load
Column ENGTYPE_1 Load
Column NL_NAME_1 Load
Column VARNAME_1 Load
Column Data_country Load
Column Data_cases Load
Column Data_recovered Load
Column Data_deaths Load
Column Data_tests Load
Column Data_hospitalized Load
Column Data_intensive_care Load
Column Data_datetime Load


In [12]:
from qgis.utils import iface


label_settings = QgsPalLayerSettings()
label_settings.drawBackground = True
fieldname_1 = "NAME_1"
fieldname_2 = "Data_tests"
#label_settings.fieldName = '''concat('BASE: ' + to_string("''' + fieldname_1 + '''"),'\nL = ' + to_string("''' + fieldname_2 + '''") + 'm')'''
#label_settings.fieldName = ('''"NAME_1" || '\n '|| "Data_tests"''')
label_settings.fieldName = "Data_tests"
label_settings.enabled = True
#label_settings.placement= QgsPalLayerSettings.Center
label_settings.bufferDraw= True
label_settings.bufferSize= 1
#label_settings.setDataDefinedProperty(QgsPalLayerSettings.Size,True,True,'8','')

text_format = QgsTextFormat()

text_format.setFont(QFont("Arial", 12))
text_format.setSize(55) 
                     
background_color = QgsTextBackgroundSettings()
text_format.setColor(QColor("red"))

background_color.setEnabled(False)
text_format.setBackground(background_color )
label_settings.setFormat(text_format)

cases_layer.setLabeling(QgsVectorLayerSimpleLabeling(label_settings))
cases_layer.setLabelsEnabled(True)
cases_layer.triggerRepaint()

Criar os limites e a graduação das cores

In [13]:
from qgis.PyQt import QtGui

myOpacity = 1

myTargetField = 'Data_tests'
myRangeList = []

myMin = 10000
myMax = 20000
myLabel = '10000-20000'
myColour = QtGui.QColor('#cce5ff')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 20000
myMax = 30000
myLabel = '20000-30000'
myColour = QtGui.QColor('#99ccff')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 30000
myMax = 40000
myLabel = '3000 - 4000'
myColour = QtGui.QColor('#66b2ff')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)


myMin = 40000
myMax = 50000
myLabel = '40000 - 50000'
myColour = QtGui.QColor('#3399ff')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 50000
myMax = 60000
myLabel = '50000 - 60000'
myColour = QtGui.QColor('#0066cc')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 60000
myMax = 70000
myLabel = '60000 - 70000'
myColour = QtGui.QColor('#004c99')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 70000
myMax = 90000
myLabel = '70000 - 90000'
myColour = QtGui.QColor('#003366')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 90000
myMax = 150000
myLabel = '90000-150000'
myColour = QtGui.QColor('#001933')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = -1
myMax = -1
myLabel = 'Sem Dados'
myColour = QtGui.QColor('#333333')
mySymbol = QgsSymbol.defaultSymbol(cases_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
myRenderer = QgsGraduatedSymbolRenderer('', myRangeList)
myClassificationMethod = QgsApplication.classificationMethodRegistry().method("EqualInterval")
myRenderer.setClassificationMethod(myClassificationMethod)
myRenderer.setClassAttribute(myTargetField)

cases_layer.setRenderer(myRenderer)

In [14]:
xt = cases_layer.extent()
width  = 1024
height = int(width*xt.height()/xt.width())

project = QgsProject().instance()
project.addMapLayers([cases_layer])
manager = project.layoutManager()
layoutName = 'confirmed_cases'
layouts_list = manager.printLayouts()
for layout in layouts_list:
    if layout.name() == layoutName:
        manager.removeLayout(layout)
        
layout = QgsPrintLayout(project)
layout.initializeDefaults()
layout.setName(layoutName)
manager.addLayout(layout)

pc = layout.pageCollection()
pc.page(0).attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map = QgsLayoutItemMap(layout)
map.setRect(20,20,20,20)
map.setLayers([cases_layer])



ms = QgsMapSettings()
ms.setLayers([cases_layer])
ms.setExtent(xt)
map.setExtent(xt)


layout.addLayoutItem(map)



map.attemptMove(QgsLayoutPoint(0,0,QgsUnitTypes.LayoutMillimeters))
map.attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map.setBackgroundColor(QColor(250,250,250))




legend = QgsLayoutItemLegend(layout)

legend.setReferencePoint(6)
legend.setScale(3)
legend.attemptMove(QgsLayoutPoint(0,height,QgsUnitTypes.LayoutMillimeters))

layout.addLayoutItem(legend)
legend.setLegendFilterByMapEnabled(True)

legend.setLinkedMap(map)

exporter = QgsLayoutExporter(manager.layoutByName("confirmed_cases"))

image = exporter.renderPageToImage(0)
image = exporter.renderPageToImage(0)
image = exporter.renderPageToImage(0)

from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
image.save('images/map_tests.png',"PNG")
Image( imgbuf.data() )